# Season 13 Election Notes

>**FORBIDDEN KNOWLEDGE WARNING**
>
>This document contains some information that is not currently present on the main site and may be considered spoilers.
>Continue at your own discretion. Some information in this document cannot be shared on the Discord without spoiler tags.

In [2]:
%matplotlib inline
from blaseball_mike.models import *
import plotly.express as plot
import plotly.io as _pio
import plotly.subplots as subplot

import os
import sys
pdir = os.path.abspath(os.path.join(os.path.dirname(''), os.path.pardir))
sys.path.append(pdir)
from display import *
from blessings import *
sys.path.remove(pdir)

_pio.renderers.default = "notebook_connected"

pies = Team.load_by_name("Philly Pies")

In [3]:
sim = SimulationData.load()
print(f"Last Updated Season {sim.season}, Day {sim.day}")

Last Updated Season 13, Day 77


---
## Decrees

Decrees are chosen by majority votes across all teams. `1` decree will pass this season.

### Bird Rights
**Allows Division Winners to Hatch a Player from the Field of Eggs with a chosen name from the Filtered Pool.**

This should modify the existing Playoff Birth, which gives teams that make the playoffs a new player in their shadows,
by letting the top team in each Division choose their new player's name. This is entirely cosmetic.

### Force Fields
**A Player on each Team is placed in the Force Field, protecting them from Steals and Swaps. Use a Will to move the
Field to another Player.**

Could be used to prevent valuable players from being stolen.

### Free Wills
**Teams that finish in the Bottom 4 of the League will receive an extra Will in that Season's Election.**

Beneficial to lower-performance teams, as it will help them catch up with additional boosts.

___
## Wills

Will are selected for each team by raffle. Each team will draw `2` random wills, where wills with more votes are more
likely to be selected.

### Plunder
**Steal a Player from the Idols board. Send a random Player from your Team back.**

Due to the random nature of the target on our team, this tests the best case (swapping the Best Player on the Idol Board
for a position with our worst player in that position) and the worst case (swaping the Best Player on the Idol Board for
a position with our best player in that position).

**Best Case Swap:**

In [4]:
idols = [x.player for x in Idol.load().values()]

idols.sort(key=lambda x: x.batting_rating, reverse=True)
best_batting_idol = idols[0]
idols.sort(key=lambda x: x.pitching_rating, reverse=True)
best_pitching_idol = idols[0]


worst_batter = sort_lineup(pies, 1)[0]
print(f"Batting: Swap {best_batting_idol.name} for {worst_batter.name}")

pandas.DataFrame(replace_player(pies, worst_batter, bat_star=best_batting_idol.batting_stars, run_star=best_batting_idol.baserunning_stars, def_star=best_batting_idol.defense_stars))

Batting: Swap Nagomi Mcdaniel for Yusef Puddles


,Pies
batting_change,+11.0063%
pitching_change,+0.0000%
baserunning_change,+7.6644%
defense_change,+2.5817%


In [5]:
worst_pitcher = sort_rotation(pies, 1)[0]
print(f"Pitching: Swap {best_pitching_idol.name} for {worst_pitcher.name}")

pandas.DataFrame(replace_player(pies, worst_pitcher, pitch_star=best_pitching_idol.pitching_stars))


Pitching: Swap Nagomi Mcdaniel for Henry Marshallow


,Pies
batting_change,+0.0000%
pitching_change,+17.4262%
baserunning_change,+0.0000%
defense_change,+0.0000%


**Worst Case Swap:**

In [6]:
best_batter = sort_lineup(pies, 1, order="best")[0]
print(f"Batting: Swap {best_batting_idol.name} for {best_batter.name}")

pandas.DataFrame(replace_player(pies, best_batter, bat_star=best_batting_idol.batting_stars, run_star=best_batting_idol.baserunning_stars, def_star=best_batting_idol.defense_stars))

Batting: Swap Nagomi Mcdaniel for Jaxon Buckley


,Pies
batting_change,+1.2015%
pitching_change,+0.0000%
baserunning_change,+6.3892%
defense_change,+6.9881%


In [7]:
best_pitcher = sort_rotation(pies, 1, order="best")[0]
print(f"Pitching: Swap {best_pitching_idol.name} for {best_pitcher.name}")

pandas.DataFrame(replace_player(pies, best_pitcher, pitch_star=best_pitching_idol.pitching_stars))


Pitching: Swap Nagomi Mcdaniel for Elvis Figueroa


,Pies
batting_change,+0.0000%
pitching_change,+7.0067%
baserunning_change,+0.0000%
defense_change,+0.0000%


### Exchange
**Swap a Player from a Team in the opposite Subleague with a Player from your Team.**

There are 139 possible Wild League players to swap with our 12 existing players, which is too many permuations for me to
list here. Instead, here's the list of the top 10 players available to choose from by star rating. For maximum impact it
is recommended we swap the player with the worst batter or pitcher respectively.

Note that position does not matter for the swap, so a player that is currently batting can still be added as a pitcher
and vice-versa.

**Best Batter options and Pies worst Batter:**

In [8]:
wild = Subleague.load("aabc11a1-81af-4036-9f18-229c759ca8a9")
all_wild = []
for team in wild.teams.values():
    all_wild.extend(team.lineup + team.rotation)
    
all_wild.sort(key=lambda x: x.batting_rating, reverse=True)
set_heatmap(get_batting_stlats(all_wild[0:10]), maxVal=1)


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Beck Whitney,1.163474,0.951409,1.431090,1.228751,1.427907,0.035629,1.164729,0.001000,0.711079
Valentine Games,1.159611,1.047322,1.213784,1.642018,1.302306,0.012096,1.515499,0.059992,1.063348
Jessica Telephone,1.129590,1.113282,1.216455,1.326294,0.982475,0.215902,1.574396,0.100000,0.919039
Knight Triumphant,1.069396,1.275457,0.882189,1.326139,1.167963,0.168635,1.118446,0.001000,0.710728
Fitzgerald Blackburn,1.044396,0.819049,1.554827,0.855190,0.993876,0.421207,0.971062,0.100000,1.473336
Jacob Haynes,1.030265,1.223815,0.902263,1.061913,1.179008,0.001000,0.356483,0.100000,0.783724
Gerund Pantheocide,1.027445,1.300367,1.034786,0.893386,0.575666,0.270360,0.571867,0.001000,0.289087
Aldon Cashmoney,1.026237,1.044073,1.122097,0.729312,1.199005,0.229409,0.764498,0.100000,0.548276
Dudley Mueller,1.009601,1.525780,0.731321,0.905664,0.964410,0.263640,0.891322,0.100000,1.513180
Margarito Nava,1.000296,0.788340,1.244494,1.244454,1.404628,0.522716,1.131332,0.038358,0.786623


In [9]:
set_heatmap(get_batting_stlats(sort_lineup(pies, 1)), maxVal=1)

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Yusef Puddles,0.429562,1.033698,0.184577,0.675196,0.151908,0.630778,0.110821,0.057271,1.010015


**Best Pitchers options and Pies worst Pitcher:**

In [10]:
all_wild.sort(key=lambda x: x.pitching_rating, reverse=True)
set_heatmap(get_pitching_stlats(all_wild[0:10]), maxVal=1)

,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Burke Gonzales,1.176989,1.171639,1.341412,0.879956,0.870391,0.973638,0.789646
Qais Dogwalker,1.105839,1.031425,1.097700,1.318705,1.112111,0.842765,0.570121
Castillo Turner,1.100450,1.115089,1.148187,0.713012,1.332078,1.379473,0.987084
Pitching Machine,1.017255,1.199024,1.106152,1.053689,0.349586,0.511801,0.714233
Rafael Davids,0.950434,0.821531,1.097610,1.057661,1.006934,1.045070,1.161455
Hiroto Wilcox,0.929726,1.048886,1.108926,0.488923,0.771052,0.826826,0.337882
Valentine Games,0.892390,0.878882,0.988182,1.175546,0.627525,0.411364,0.943146
Knight Triumphant,0.891070,1.185525,0.591861,1.051436,1.169285,0.575927,0.453896
Rigby Friedrich,0.888575,1.072799,0.939847,0.427646,1.074809,0.718857,0.537667
Fitzgerald Blackburn,0.865100,0.849077,0.980479,0.982413,0.738350,0.411036,0.775065


In [11]:
set_heatmap(get_pitching_stlats(sort_rotation(pies, 1)), maxVal=1)

,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.428691,0.233266,0.882016,1.123135,0.484581,0.571587,0.656728


### Foreshadow
**Swap a Player from your Shadows to your active roster.**

There are 14 shadows players to swap with our 12 existing players. Below is a breakdown comparing the best shadows
players with the worst player in the particular position.

Note that position does not matter for the swap: any shadows player can bat or pitch.


**Batting:**

In [12]:
shadows = pies.bullpen + pies.bench
shadows.sort(key=lambda x: x.batting_rating, reverse=True)
best_batting_shadows = shadows[0:4]
worst_batter = sort_lineup(pies, 1)[0]

get_stars(best_batting_shadows + [worst_batter])

,Batting,Pitching,Baserunning,Defense
Haruta Byrd,3.5,2.5,2.5,1.5
Javier Lotus,3.5,1.0,3.5,4.0
August Obrien,3.5,1.0,3.5,2.0
Jefferson Delacruz,3.0,0.5,3.5,4.0
Yusef Puddles,2.0,2.5,3.0,3.5


In [13]:
set_heatmap(get_batting_stlats(best_batting_shadows + [worst_batter]), maxVal=1)

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Haruta Byrd,0.724391,0.721433,0.917077,0.853010,0.585921,0.832203,0.169263,0.100000,0.124161
Javier Lotus,0.682621,0.938964,0.386700,0.901328,0.942211,0.205027,0.884526,0.100000,0.815592
August Obrien,0.681669,0.641217,0.927569,0.913891,0.492708,0.923854,0.556849,0.100000,0.463958
Jefferson Delacruz,0.595750,0.479940,0.643757,0.903986,0.537770,0.503675,0.434424,0.100000,0.671689
Yusef Puddles,0.429562,1.033698,0.184577,0.675196,0.151908,0.630778,0.110821,0.057271,1.010015


**Pitching:**

In [14]:
shadows.sort(key=lambda x: x.pitching_rating, reverse=True)
best_pitching_shadows = shadows[0:4]
worst_pitcher = sort_rotation(pies, 1)[0]

get_stars(best_pitching_shadows + [worst_pitcher])

,Batting,Pitching,Baserunning,Defense
Lucy Tokkan,2.0,3.5,1.0,3.0
Twofurious Puddles,2.5,3.5,2.0,3.5
Case Lancaster,0.0,3.5,2.0,3.0
Tiana Takahashi,1.0,3.0,2.5,3.0
Henry Marshallow,2.0,2.0,1.5,3.5


In [15]:
set_heatmap(get_pitching_stlats(best_pitching_shadows + [worst_pitcher]), maxVal=1)

,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Lucy Tokkan,0.741748,0.901966,0.995562,0.263758,0.973020,0.180980,0.321805
Twofurious Puddles,0.694922,0.811110,0.650034,0.807712,0.718615,0.416240,0.725894
Case Lancaster,0.658365,0.867044,0.829197,0.190119,0.825333,0.868251,0.082117
Tiana Takahashi,0.618850,0.910065,0.961592,0.716677,0.028954,0.596122,0.945940
Henry Marshallow,0.428691,0.233266,0.882016,1.123135,0.484581,0.571587,0.656728


### Infuse
**Boost a Player on your Team by 15% to 30%.**

See below for how this would affect various team members at the 2 extreme percentages. This will only affect one player.

**15% Change:**

In [16]:
infuse_lineup_table, _, _ = improve_team_batting_table(pies, 0.15)
infuse_lineup_table

,old_batting_stars,new_batting_stars,change_in_batting_stars
Yusef Puddles,2.147811,3.068746,0.920935
Beasley Day,2.695709,3.559653,0.863944
Lang Richardson,2.914378,3.671442,0.757064
-od- Preston,3.691847,4.482977,0.791131
Huber Frumple,4.007441,4.764912,0.757471
Eduardo Woodman,4.165300,4.885272,0.719972
Jaxon Buckley,5.579461,6.233656,0.654195


In [17]:
set_heatmap(get_batting_stlats(improve_team_batting(pies, 0.15)), maxVal=1.0)

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Yusef Puddles,0.613749,1.183698,0.334577,0.825196,0.301908,0.480778,0.260821,0.010000,1.160015
Beasley Day,0.711931,0.835000,0.838801,0.227669,0.358479,0.318964,0.673706,0.010000,0.903672
Lang Richardson,0.734288,0.920859,0.503631,0.708549,1.028068,0.096647,0.577515,0.010000,0.636870
-od- Preston,0.896595,1.009775,1.032217,0.677863,0.291277,0.010000,0.933390,0.010000,0.219496
Huber Frumple,0.952982,0.611308,1.358875,1.067125,1.377816,0.154893,0.834028,0.010000,1.242836
Eduardo Woodman,0.977054,1.169391,0.996540,0.800666,0.725519,0.445517,0.724554,0.010000,0.855869
Jaxon Buckley,1.246731,1.440397,0.997503,1.629805,1.822769,0.010000,1.879523,0.010000,1.296485


In [18]:
infuse_rotation_table, _, _ = improve_team_pitching_table(pies, 0.15)
infuse_rotation_table


,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Henry Marshallow,2.143457,3.080575,0.937118
Nicholas Mora,2.832423,3.761138,0.928715
Doc Anice,2.841172,3.734213,0.893041
Ruslan Greatness,3.683850,4.551704,0.867854
Elvis Figueroa,4.748317,5.651779,0.903462


In [19]:
set_heatmap(get_pitching_stlats(improve_team_pitching(pies, 0.15)), maxVal=1)


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.616115,0.383266,1.032016,1.273135,0.634581,0.721587,0.806728
Nicholas Mora,0.752228,1.290948,0.493723,0.703609,0.517345,0.632330,0.581032
Doc Anice,0.746843,1.078114,0.509045,0.986037,0.606020,0.748965,0.294698
Ruslan Greatness,0.910341,0.844663,0.870262,1.340695,1.101602,0.737925,1.521838
Elvis Figueroa,1.130356,1.186387,1.165972,1.080460,0.839138,0.478725,0.742661


**30% Change:**

In [20]:
infuse_lineup_table, _, _ = improve_team_batting_table(pies, 0.30)
infuse_lineup_table

,old_batting_stars,new_batting_stars,change_in_batting_stars
Yusef Puddles,2.147811,3.885398,1.737587
Beasley Day,2.695709,4.319837,1.624128
Lang Richardson,2.914378,4.386193,1.471815
-od- Preston,3.691847,5.182973,1.491126
Huber Frumple,4.007441,5.493492,1.486051
Eduardo Woodman,4.165300,5.585918,1.420618
Jaxon Buckley,5.579461,6.873935,1.294474


In [21]:
set_heatmap(get_batting_stlats(improve_team_batting(pies, 0.30)), maxVal=1)

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Yusef Puddles,0.777080,1.333698,0.484577,0.975196,0.451908,0.330778,0.410821,0.010000,1.310015
Beasley Day,0.863967,0.985000,0.988801,0.377669,0.508479,0.168964,0.823706,0.010000,1.053672
Lang Richardson,0.877239,1.070859,0.653631,0.858549,1.178068,0.010000,0.727515,0.010000,0.786870
-od- Preston,1.036595,1.159775,1.182217,0.827863,0.441277,0.010000,1.083390,0.010000,0.369496
Huber Frumple,1.098698,0.761308,1.508875,1.217125,1.527816,0.010000,0.984028,0.010000,1.392836
Eduardo Woodman,1.117184,1.319391,1.146540,0.950666,0.875519,0.295517,0.874554,0.010000,1.005869
Jaxon Buckley,1.374787,1.590397,1.147503,1.779805,1.972769,0.010000,2.029523,0.010000,1.446485


In [22]:
infuse_rotation_table, _, _ = improve_team_pitching_table(pies, 0.30)
infuse_rotation_table


,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Henry Marshallow,2.143457,4.003592,1.860134
Nicholas Mora,2.832423,4.690645,1.858222
Doc Anice,2.841172,4.636189,1.795017
Ruslan Greatness,3.683850,5.441981,1.758131
Elvis Figueroa,4.748317,6.571009,1.822692


In [23]:
set_heatmap(get_pitching_stlats(improve_team_pitching(pies, 0.30)), maxVal=1)

,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.800718,0.533266,1.182016,1.423135,0.784581,0.871587,0.956728
Nicholas Mora,0.938129,1.440948,0.643723,0.853609,0.667345,0.782330,0.731032
Doc Anice,0.927238,1.228114,0.659045,1.136037,0.756020,0.898965,0.444698
Ruslan Greatness,1.088396,0.994663,1.020262,1.490695,1.251602,0.887925,1.671838
Elvis Figueroa,1.314202,1.336387,1.315972,1.230460,0.989138,0.628725,0.892661


### Trust
**Receive a mystery Benefit for one of your Teams' Players.**

The Benifit here is unknown. It is likely either a stat increase or an unknown Modification.

### Alternate
**Call in an Alternate for a Player on your Team.**

Alternates are when players stats are randomized. Below shows the change in Team Average Stars from rerolling our worst
player, assuming they would be replaced with the Statistically Average player.

In [24]:
worst_pie = sort_overall(pies, 1)[0]

print(f"Rerolling {worst_pie.name}")
if worst_pie in pies.rotation:
    out = replace_player(pies, worst_pie, pitch_star=NEW_PLAYER["pitching"])
else:
    out = replace_player(pies, worst_pie, bat_star=NEW_PLAYER["batting"], run_star=NEW_PLAYER["baserunning"], def_star=NEW_PLAYER["defense"])
pandas.DataFrame(out)

Rerolling Henry Marshallow


,Pies
batting_change,+0.0000%
pitching_change,-2.5738%
baserunning_change,+0.0000%
defense_change,+0.0000%


### Revoke
**Revoke a Player from your roster.**

Revoking will cause the player to change teams, potentially every season. This will cause our other players to play more
often, increasing our average team performance if we chose the worst player. See below for analysis of removing our
worst Batter or Pitcher.

**Batting:**

In [25]:
print(f'Removing {worst_batter.name}')
pandas.DataFrame(remove_player(pies, worst_batter))

Removing Yusef Puddles


,Pies
batting_change,+4.8416%
pitching_change,+0.0000%
baserunning_change,+1.6607%
defense_change,-2.7701%


**Pitching:**

In [26]:
print(f'Removing {worst_pitcher.name}')
pandas.DataFrame(remove_player(pies, worst_pitcher))


Removing Henry Marshallow


,Pies
batting_change,+0.0000%
pitching_change,+5.5319%
baserunning_change,+0.0000%
defense_change,+0.0000%


### Enrich
**Pick a modification for your Team next season.**

This will gives the option of choosing between
* The Fourth Strike: Those with the Fourth Strike will get an extra strike in each at bat.
* Shame Donor: A Team or Player who adds Unruns to a Shamed opponents' future pile.
* Earlybirds: This team will Overperform in the Earlseason.
* Late to the Party: This team will Overperform in the Lateseason.

Seasonal Modifications have been mixed in the past. For Fourth Strike Analysis, see the appropriate
[SIBR Paper](https://research.blaseball-reference.com/Fourth_Strike.pdf). Earlybirds or Late to the Party would be
roughly equivelant to a 5% boost overall.

___
## Blessings

Blessings are selected by raffle across all teams. Each blessing will be awarded to a random team, where teams with more
votes are more likely to be selected.

### Rise and Grind
**Rise to the top. The worst pitcher on your Team pays out 5x more for Snacks.**

This would increase the likelyhood that our worst pitcher would be highly idoled, and idol board shenanigans have
previously been bad overall. See below for the performance statistics for this player last season and an estimation of
overall payout.

In [27]:
rise_and_grind_player = worst_pitcher
rise_and_grind = get_pitching_stats(rise_and_grind_player, season=12)
rise_and_grind

,games,wins,losses,win_pct,pitches_thrown,batters_faced,outs_recorded,innings,runs_allowed,shutouts,quality_starts,strikeouts,walks,home_runs_allowed,hits_allowed,hit_by_pitches,earned_run_average,walks_per_9,hits_per_9,strikeouts_per_9,home_runs_per_9,whip,strikeouts_per_walk
Henry Marshallow,20,19,1,0.95,2039,690,553,184.1,51,3,13,234,19,15,131,0,2.49,0.93,6.40,11.42,0.73,0.904,12.32


In [28]:
strikeouts = rise_and_grind["strikeouts"]
shutouts = rise_and_grind["shutouts"]
payout = int(shutouts)*12500 + int(strikeouts)*200
new_payout = payout*5
print("Normal payout with max upgrades: {:,}".format(payout))
print("Boosted payout with max upgrades: {:,}".format(new_payout))
best = 7*12500 + 351*200 # Burke Gonzales
print("Payout of most profitable pitcher in S12: {:,}".format(best))

Normal payout with max upgrades: 84,300
Boosted payout with max upgrades: 421,500
Payout of most profitable pitcher in S12: 157,700


### Center of Attention
**Look at me. Most Idolized player on your team gives double-payouts as an Idol.**

This would likely improve the payout on Jaxon Buckley. See below for the performance statistics for Jaxon last
season, and an estimation of overall payout.

In [29]:
jaxon = Player.find_by_name("Jaxon Buckley")
center_of_attention = get_batting_stats(jaxon, season=12)
center_of_attention

,appearances,batting_average,on_base_percentage,slugging,plate_appearances,at_bats,hits,walks,singles,doubles,triples,quadruples,home_runs,runs_batted_in,strikeouts,sacrifice_bunts,sacrifice_flies,at_bats_risp,hits_risp,batting_average_risp,on_base_slugging,total_bases,hit_by_pitches,ground_outs,flyouts,gidp
Jaxon Buckley,99,0.331,0.365,0.657,539,496,164,31,75,46,13,0,30,109,46,7,0,94,40,0.426,1.022,326,0,175,94,3


In [30]:
homers = int(center_of_attention["home_runs"])
hits = int(center_of_attention["hits"]) - homers
payout = homers*4000 + hits*1500
new_payout = payout*2
print("Normal payout with max upgrades: {:,}".format(payout))
print("Boosted payout with max upgrades: {:,}".format(new_payout))
print("Payout of most profitable batter in S12: 730,000")

Normal payout with max upgrades: 321,000
Boosted payout with max upgrades: 642,000
Payout of most profitable batter in S12: 730,000


### Sun Dialed & Unholey
**Boosts a random player on your Team by 1% anytime you activate Sun 2.**

**The Black Hole will impair a random player on the other team by 1% anytime your Team activates it.**

~Random~ target. See below for weather and boost analysis.

In [31]:
pies_games = Game.load_by_season(season=0, team_id=pies.id)
sun2_games = [x for x in pies_games.values() if f"Sun 2 set a Win upon the {pies.nickname}" in x.outcomes or f"Sun 2 set a Win upon the {pies.nickname}." in x.outcomes]
print(f"Sun 2 has activated for the {pies.nickname} {len(sun2_games)} times in league history.")

Sun 2 has activated for the Pies 3 times in league history.


In [32]:
total_blackholes = 0
for game in pies_games.values():
    for outcome in game.outcomes:
        if "The Black Hole swallowed a Win" in outcome and pies.nickname not in outcome:
            total_blackholes += 1
print(f"Black Hole has activated for the {pies.nickname} {total_blackholes} times league history.")

Black Hole has activated for the Pies 6 times league history.


In [33]:
table, total, avg = improve_team_overall_table(pies, 0.01)
table

,old_batting_stars,old_pitching_stars,old_baserunning_stars,old_defense_stars,new_batting_stars,new_pitching_stars,new_baserunning_stars,new_defense_stars,change_in_batting_stars,change_in_pitching_stars,change_in_baserunning_stars,change_in_defense_stars
Yusef Puddles,2.147811,2.353819,2.817447,3.596398,2.215192,2.408734,2.869284,3.640719,0.067381,0.054915,0.067381,0.044321
Beasley Day,2.695709,1.553222,2.470609,1.617475,2.761565,1.607525,2.521358,1.684328,0.065856,0.054302,0.065856,0.066852
Lang Richardson,2.914378,0.259236,3.241335,3.451126,2.966349,0.366987,3.290677,3.497818,0.051971,0.107751,0.051971,0.046692
-od- Preston,3.691847,1.380646,3.687356,1.957512,3.749340,1.435649,3.735299,2.017598,0.057494,0.055003,0.057494,0.060086
Huber Frumple,4.007441,2.185193,4.209709,3.126600,4.058762,2.238573,4.255711,3.171558,0.051321,0.053379,0.051321,0.044958
Eduardo Woodman,4.165300,1.891005,3.519368,3.554369,4.214065,1.946350,3.566968,3.597999,0.048765,0.055345,0.048765,0.043631
Jaxon Buckley,5.579461,2.397229,3.263785,2.054179,5.623573,2.451348,3.312447,2.110535,0.044111,0.054118,0.044111,0.056357
Henry Marshallow,1.903035,2.143457,1.475110,3.576140,1.977467,2.207197,1.538863,3.625495,0.074432,0.063740,0.074432,0.049355
Nicholas Mora,4.619012,2.832423,4.360111,4.746472,4.668723,2.894612,4.407113,4.783404,0.049711,0.062189,0.049711,0.036932
Doc Anice,2.423112,2.841172,1.711568,3.683109,2.476967,2.900636,2.067616,3.724647,0.053855,0.059464,0.053855,0.041538


### High Pressure
**Your Team plays 25% better when Runners are On Base in Flooding Weather.**

See below for player change and weather analysis.

In [34]:
print("Weather stats for Season 13")
games = Game.load_by_season(season=13)
weathers = pandas.Series([x.weather.text for x in games.values()])
table = pandas.DataFrame(weathers.value_counts(), columns=["weather_count"])
table["weather_percent"] = (table["weather_count"]/len(games))*100
table

Weather stats for Season 13


,weather_count,weather_percent
Sun 2,199,16.750842
Flooding,196,16.498316
Black Hole,180,15.151515
Birds,141,11.868687
Solar Eclipse,133,11.195286
Peanuts,132,11.111111
Reverb,72,6.060606
Feedback,60,5.050505
Blooddrain,47,3.956229
Coffee 3s,15,1.262626


In [35]:
table, total, avg = improve_team_overall_table(pies, 0.25)
table

,old_batting_stars,old_pitching_stars,old_baserunning_stars,old_defense_stars,new_batting_stars,new_pitching_stars,new_baserunning_stars,new_defense_stars,change_in_batting_stars,change_in_pitching_stars,change_in_baserunning_stars,change_in_defense_stars
Yusef Puddles,2.147811,2.353819,2.817447,3.596398,3.620152,3.758554,4.051608,4.614741,1.472341,1.404735,1.472341,1.018343
Beasley Day,2.695709,1.553222,2.470609,1.617475,4.071672,2.927222,3.695814,2.975459,1.375963,1.374000,1.375963,1.357983
Lang Richardson,2.914378,0.259236,3.241335,3.451126,4.156594,1.877300,4.436522,4.510131,1.242215,1.618064,1.242215,1.059006
-od- Preston,3.691847,1.380646,3.687356,1.957512,4.953990,2.764807,4.857175,3.221024,1.262144,1.384162,1.262144,1.263512
Huber Frumple,4.007441,2.185193,4.209709,3.126600,5.253992,3.560293,5.343852,4.168073,1.246551,1.375100,1.246551,1.041473
Eduardo Woodman,4.165300,1.891005,3.519368,3.554369,5.353766,3.303215,4.685600,4.567511,1.188467,1.412210,1.188467,1.013143
Jaxon Buckley,5.579461,2.397229,3.263785,2.054179,6.661884,3.785767,4.450096,3.273476,1.082423,1.388538,1.082423,1.219298
Henry Marshallow,1.903035,2.143457,1.475110,3.576140,3.432429,3.695996,2.872199,4.653350,1.529394,1.552538,1.529394,1.077210
Nicholas Mora,4.619012,2.832423,4.360111,4.746472,5.758261,4.380123,5.511371,5.630958,1.139249,1.547700,1.139249,0.884486
Doc Anice,2.423112,2.841172,1.711568,3.683109,3.703675,4.334151,3.975775,4.657734,1.280563,1.492979,1.280563,0.974626


### Peanut Butter
**One Player on your Team becomes Chunky, one becomes Smooth.**

Chunky and Smooth are unknown, though likely related to Peanuts or Peanut weather.

### Spare Flippers
**Two Random players in your Team's lineup get Flippers.**

Flippers is a player modification that states "When immateria floods, this Runner will score instead of being cleared
from the Bases." Flood percentages can be seen in the High Pressure section.

### Stow Away
**Hatch a rookie from the Field of Eggs to join your Rotation.**

Adding a new player would increase our pitcher count to 6, see below for change in Team average stars assuming the
Statistical Average Player.

In [36]:
pandas.DataFrame(add_player(pies, pitch_star=NEW_PLAYER["pitching"], def_star=NEW_PLAYER["defense"]))

,Pies
batting_change,+0.0000%
pitching_change,-5.8328%
baserunning_change,+0.0000%
defense_change,-0.6634%


### Pitching Flotation Bubble
**Your entire Division's pitching is boosted by 4%.**

See below for team and division improvement.

In [37]:
table, _, _ = improve_team_pitching_table(pies, 0.04)
pandas.DataFrame(table)

,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Henry Marshallow,2.143457,2.396875,0.253418
Nicholas Mora,2.832423,3.080794,0.248371
Doc Anice,2.841172,3.078983,0.237811
Ruslan Greatness,3.683850,3.912942,0.229092
Elvis Figueroa,4.748317,4.987815,0.239498


In [38]:
mild_high = Division.load_by_name("mild high")
pitch_bubble = pandas.DataFrame()
for team in mild_high.teams.values():
    _, _, average = improve_team_pitching_table(team, 0.04)
    average = average.rename(team.nickname)
    pitch_bubble = pitch_bubble.append(average)
pitch_bubble.transpose()

,Millennials,Steaks,Lovers,Pies,Garages,Mechanics
change_in_pitching_stars,0.236241,0.234046,0.236649,0.241638,0.227859,0.239808
new_pitching_stars,3.983566,2.836437,3.518286,3.491482,2.925830,3.865248
old_pitching_stars,3.747324,2.602390,3.281637,3.249844,2.697971,3.625440


In [39]:
fig = subplot.make_subplots(cols=1, rows=1)
fig.add_bar(x=pitch_bubble.index, y=pitch_bubble["change_in_pitching_stars"], col=1, row=1)
fig.update_layout(showlegend=False, title_text="Pitching Flotation Bubble: Change in Average Stars")
fig.show()

### Hitting Flotation Bubble
**Your entire Division's hitting is boosted by 4%.**

See below for team and division improvement.

In [40]:
table, _, _ = improve_team_batting_table(pies, 0.04)
pandas.DataFrame(table)

,old_batting_stars,new_batting_stars,change_in_batting_stars
Yusef Puddles,2.147811,2.410803,0.262992
Beasley Day,2.695709,2.948039,0.252330
Lang Richardson,2.914378,3.120367,0.205989
-od- Preston,3.691847,3.917181,0.225335
Huber Frumple,4.007441,4.212686,0.205245
Eduardo Woodman,4.165300,4.359810,0.194510
Jaxon Buckley,5.579461,5.755454,0.175993


In [41]:
hit_bubble = pandas.DataFrame()
for team in mild_high.teams.values():
    _, _, average = improve_team_batting_table(team, 0.04)
    average = average.rename(team.nickname)
    hit_bubble = hit_bubble.append(average)
hit_bubble.transpose()

,Millennials,Steaks,Lovers,Pies,Garages,Mechanics
change_in_batting_stars,0.218991,0.204995,0.215483,0.217485,0.209039,0.262990
new_batting_stars,3.747908,3.850068,2.831042,3.817763,3.653414,2.484964
old_batting_stars,3.528917,3.645073,2.615559,3.600278,3.444375,2.221974


In [42]:
fig = subplot.make_subplots(cols=1, rows=1)
fig.add_bar(x=hit_bubble.index, y=hit_bubble["change_in_batting_stars"], col=1, row=1)
fig.update_layout(showlegend=False, title_text="Hitting Flotation Bubble: Change in Average Stars")
fig.show()

### Defense Flotation Bubble
**Your entire Division's defense is boosted by 4%.**

See below for team and division improvement.

In [43]:
table, _, _ = improve_team_defense_table(pies, 0.04)
pandas.DataFrame(table)

,old_defense_stars,new_defense_stars,change_in_defense_stars
Yusef Puddles,3.596398,3.771363,0.174965
Beasley Day,1.617475,1.874712,0.257237
Lang Richardson,3.451126,3.635072,0.183946
-od- Preston,1.957512,2.190466,0.232954
Huber Frumple,3.126600,3.304417,0.177817
Eduardo Woodman,3.554369,3.726999,0.172630
Jaxon Buckley,2.054179,2.274114,0.219935
Henry Marshallow,3.576140,3.768800,0.192661
Nicholas Mora,4.746472,4.893326,0.146854
Doc Anice,3.683109,3.847719,0.164610


In [44]:
def_bubble = pandas.DataFrame()
for team in mild_high.teams.values():
    _, _, average = improve_team_defense_table(team, 0.04)
    average = average.rename(team.nickname)
    def_bubble = def_bubble.append(average)
def_bubble.transpose()

,Millennials,Steaks,Lovers,Pies,Garages,Mechanics
change_in_defense_stars,0.250979,0.197875,0.209982,0.186035,0.267027,0.323440
new_defense_stars,3.071150,3.474153,3.486752,3.517955,2.992430,2.875985
old_defense_stars,2.820172,3.276278,3.276770,3.331920,2.725403,2.552545


In [45]:
fig = subplot.make_subplots(cols=1, rows=1)
fig.add_bar(x=def_bubble.index, y=def_bubble["change_in_defense_stars"], col=1, row=1)
fig.update_layout(showlegend=False, title_text="Defense Flotation Bubble: Change in Average Stars")
fig.show()

### Baserunning Flotation Bubble
**Your entire Division's baserunning is boosted by 4%.**

See below for team and division improvement.

In [46]:
table, _, _ = improve_team_baserunning_table(pies, 0.04)
pandas.DataFrame(table)

,old_baserunning_stars,new_baserunning_stars,change_in_baserunning_stars
Yusef Puddles,2.817447,3.023073,0.205626
Beasley Day,2.470609,2.672526,0.201916
Lang Richardson,3.241335,3.437737,0.196403
-od- Preston,3.687356,3.878432,0.191076
Huber Frumple,4.209709,4.393368,0.183660
Eduardo Woodman,3.519368,3.709222,0.189854
Jaxon Buckley,3.263785,3.457718,0.193933


In [47]:
run_bubble = pandas.DataFrame()
for team in mild_high.teams.values():
    _, _, average = improve_team_baserunning_table(team, 0.04)
    average = average.rename(team.nickname)
    run_bubble = run_bubble.append(average)
run_bubble.transpose()

,Millennials,Steaks,Lovers,Pies,Garages,Mechanics
change_in_baserunning_stars,0.208898,0.199959,0.236322,0.194638,0.207736,0.251661
new_baserunning_stars,3.581181,3.829103,3.099301,3.510297,3.405116,2.101475
old_baserunning_stars,3.372282,3.629143,2.862979,3.315658,3.197381,1.849814


In [48]:
fig = subplot.make_subplots(cols=1, rows=1)
fig.add_bar(x=run_bubble.index, y=run_bubble["change_in_baserunning_stars"], col=1, row=1)
fig.update_layout(showlegend=False, title_text="Baserunning Flotation Bubble: Change in Average Stars")
fig.show()

---
## Appendix
* [Description of Attributes](https://www.blaseball.wiki/w/Player_Attributes)
* [Stlat Viewer](https://slavfox.space/abslve/?foreboding-kaleidoscope#PHIL)
* [Historical Player Graphs](http://yoori.space/hloroscopes/)
